In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
gaze_df = pd.read_csv('gaze_log.csv')

In [7]:
gaze_df.head

<bound method NDFrame.head of     left_eye_from_center right_eye_from_center  time_start  time_end  \
0            [0.0, -1.5]          [-1.0, -1.0]    1.959574  2.537568   
1           [-1.0, -1.5]           [-2.0, 0.0]    2.537568  2.602491   
2            [0.5, -1.5]           [0.0, -1.0]    2.602491  2.649572   
3            [0.5, -1.5]           [0.0, -1.0]    2.649572  2.697045   
4           [-0.5, -1.5]           [1.0, -1.5]    2.697045  2.744644   
..                   ...                   ...         ...       ...   
113         [-3.0, -1.0]           [0.0, -0.5]    8.073423  8.119355   
114         [-3.0, -1.0]           [0.0, -0.5]    8.119355  8.165853   
115         [-3.5, -1.0]           [1.0, -1.0]    8.165853  8.212563   
116         [-3.5, -1.0]           [1.0, -1.0]    8.212563  8.259641   
117         [-3.0, -1.0]           [1.5, -1.5]    8.259641  8.306438   

    left_eye_dim right_eye_dim  
0       (27, 44)      (26, 40)  
1       (27, 43)      (26, 38)  
2     

Eye position:
Where the pupil is relative to the whole eye |---o---| = 50%

In [ ]:
gaze_df['Right_eye_pos'] = ga